# Find best model

### Imports

In [51]:
import __main__ as main

from helpers.training import *
from pathlib import Path
from joblib import load
from sklearn.model_selection import train_test_split
from datetime import datetime
from helpers.is_interactive import is_interactive

### Run dependency notebooks

In [52]:
if is_interactive(main):
    %run 01_0_data_wrangling.ipynb -p
    %run 01_2_data_wrangling_kaggle.ipynb -p
    %run 02_0_scaling.ipynb -p
    %run 02_1_scaling_kaggle.ipynb -p

Running previous notebooks...


### Load Dataframe

In [53]:
source_path = Path('./data/model/02_train_data.pkl')
training_data = load(source_path)

df = training_data['dataset']

X = df.drop('price_cleaned', axis=1)
y = df['price_cleaned']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

df.head(10)

Living space  Plot area  price_cleaned     Floor  ForestDensityL  \
0      0.011759   0.002511      1150000.0  0.285714        0.569894   
1      0.018764   0.000633      1420000.0  0.250000        0.569894   
2      0.010883   0.002511       720000.0  0.214286        0.182127   
3      0.018514   0.001054      1430000.0  0.142857        0.569894   
4      0.017013   0.000316       995000.0  0.142857        0.372216   
5      0.023017   0.003029      2160000.0  0.142857        0.212473   
6      0.014761   0.000570       550000.0  0.250000        0.569894   
7      0.012259   0.000316       590000.0  0.250000        0.057230   
8      0.008631   0.000316       547000.0  0.178571        0.308985   
9      0.013010   0.006030      1125000.0  0.142857        0.156497   

   ForestDensityM  ForestDensityS  Latitude  Longitude  \
0        0.286608        0.090908  0.808677   0.477811   
1        0.286608        0.090908  0.808677   0.477811   
2        0.095930        0.001911  0.799258   0.468164   
3        0.286608        0.090908  0.808677   0.477811   
4        0.279429        0.145835  0.803051   0.470341   
5        0.162927        0.034759  0.801165   0.461133   
6        0.286608        0.090908  0.808677   0.477811   
7        0.000000        0.000000  0.794885   0.467948   
8        0.438584        0.327528  0.801046   0.474388   
9        0.140161        0.156650  0.798179   0.461295   

   NoisePollutionRailwayL  ...  gde_workers_total_2  gde_workers_total_3  \
0                0.000000  ...         4.489742e-07         3.008376e-10   
1                0.000000  ...         4.489742e-07         3.008376e-10   
2                0.000000  ...         5.103043e-03         3.645388e-04   
3                0.000000  ...         4.489742e-07         3.008376e-10   
4                0.000000  ...         8.152190e-06         2.327617e-08   
5                0.000000  ...         4.101045e-06         8.305040e-09   
6                0.000000  ...         4.489742e-07         3.008376e-10   
7                0.187309  ...         5.103043e-03         3.645388e-04   
8                0.088907  ...         5.103043e-03         3.645388e-04   
9                0.000000  ...         4.101045e-06         8.305040e-09   

   gde_workers_total_4  gde_workers_total_5  gde_workers_total_6   rooms_2  \
0         2.015778e-13         1.350683e-16         9.050323e-20  0.020408   
1         2.015778e-13         1.350683e-16         9.050323e-20  0.020408   
2         2.604104e-05         1.860257e-06         1.328885e-07  0.020408   
3         2.015778e-13         1.350683e-16         9.050323e-20  0.020408   
4         6.645820e-11         1.897517e-13         5.417799e-16  0.020408   
5         1.681857e-11         3.405934e-14         6.897369e-17  0.020408   
6         2.015778e-13         1.350683e-16         9.050323e-20  0.020408   
7         2.604104e-05         1.860257e-06         1.328885e-07  0.020408   
8         2.604104e-05         1.860257e-06         1.328885e-07  0.020408   
9         1.681857e-11         3.405934e-14         6.897369e-17  0.020408   

    rooms_3   rooms_4   rooms_5   rooms_6  
0  0.002915  0.000416  0.000059  0.000008  
1  0.002915  0.000416  0.000059  0.000008  
2  0.002915  0.000416  0.000059  0.000008  
3  0.002915  0.000416  0.000059  0.000008  
4  0.002915  0.000416  0.000059  0.000008  
5  0.002915  0.000416  0.000059  0.000008  
6  0.002915  0.000416  0.000059  0.000008  
7  0.002915  0.000416  0.000059  0.000008  
8  0.002915  0.000416  0.000059  0.000008  
9  0.002915  0.000416  0.000059  0.000008  

[10 rows x 310 columns]

## Train all Models

In [54]:
TRAINING_FUNCTIONS = [
    train_gradient_boosting_v1,
    #train_gradient_boosting,
    #train_gradient_boosting_robust,
    #train_random_forest,
    #train_linear_regression,
    #train_mlp_regressor,
]

In [55]:
results = [
    func(X_train, X_test, y_train, y_test)
    for func in TRAINING_FUNCTIONS
]

Training GradientBoostingRegressor with -1 jobs
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [56]:
pd.DataFrame(results)

num_columns     score                                              model
0          309  0.254579  GridSearchCV(cv=5, estimator=GradientBoostingR...

In [57]:
best_model_row = results[0]
best_model = best_model_row['model']

In [58]:
best_model.best_params_

{'loss': 'absolute_error',
 'max_depth': 21,
 'max_features': 'sqrt',
 'min_samples_leaf': 13,
 'min_samples_split': 1,
 'n_estimators': 130,
 'random_state': 42}

In [59]:
best_model.best_score_

-0.2650349914469722

In [60]:
validation_data = load(Path('data/kaggle/02_model_data.pkl'))
validation_df = validation_data['dataset']
base_imputer = validation_data['base_imputer']
scaler = validation_data['scaler']
validation_df.head()

Living space  Plot area     Floor  ForestDensityL  ForestDensityM  \
0      0.026770   0.002089  0.142857        0.183264        0.100085   
1      0.028021   0.002000  0.142857        0.290820        0.170527   
2      0.015637   0.000373  0.142857        0.483981        0.358180   
3      0.016763   0.000587  0.250000        0.165213        0.076652   
4      0.018764   0.000633  0.250000        0.569894        0.286608   

   ForestDensityS  Latitude  Longitude  NoisePollutionRailwayL  \
0        0.063548  0.786382   0.475555                0.014734   
1        0.083253  0.786102   0.474934                0.010139   
2        0.125505  0.807571   0.477015                0.000000   
3        0.000000  0.787002   0.475789                0.020076   
4        0.090908  0.808677   0.477811                0.000000   

   NoisePollutionRailwayM  ...  gde_workers_total_2  gde_workers_total_3  \
0                     0.0  ...         1.242706e-04         1.385328e-06   
1                     0.0  ...         1.242706e-04         1.385328e-06   
2                     0.0  ...         4.489742e-07         3.008376e-10   
3                     0.0  ...         1.242706e-04         1.385328e-06   
4                     0.0  ...         4.489742e-07         3.008376e-10   

   gde_workers_total_4  gde_workers_total_5  gde_workers_total_6   rooms_2  \
0         1.544318e-08         1.721555e-10         1.919134e-12  0.020408   
1         1.544318e-08         1.721555e-10         1.919134e-12  0.020408   
2         2.015778e-13         1.350683e-16         9.050323e-20  0.020408   
3         1.544318e-08         1.721555e-10         1.919134e-12  0.020408   
4         2.015778e-13         1.350683e-16         9.050323e-20  0.020408   

    rooms_3   rooms_4   rooms_5   rooms_6  
0  0.002915  0.000416  0.000059  0.000008  
1  0.002915  0.000416  0.000059  0.000008  
2  0.002915  0.000416  0.000059  0.000008  
3  0.002915  0.000416  0.000059  0.000008  
4  0.002915  0.000416  0.000059  0.000008  

[5 rows x 309 columns]

In [61]:
validation_df.isna().sum()

Living space      0
Plot area         0
Floor             0
ForestDensityL    0
ForestDensityM    0
                 ..
rooms_2           0
rooms_3           0
rooms_4           0
rooms_5           0
rooms_6           0
Length: 309, dtype: int64

In [62]:
predictions = pd.DataFrame({
    'Id': list(validation_df.index),
    'Expected': best_model.predict(validation_df)
})
predictions.head()

Id      Expected
0   0  1.428903e+06
1   1  1.327381e+06
2   2  9.533572e+05
3   3  9.894934e+05
4   4  1.173226e+06

In [63]:
def getClassName(obj):
   return type(obj).__name__

def getFormattedDate(date = datetime.now()):
    return date.strftime('%Y%m%d_%H%M')

In [72]:
filename = f'{getFormattedDate()}_{getClassName(best_model.estimator)}_{getClassName(scaler)}_{getClassName(base_imputer)}_{best_model_row["score"]:.3f}'
path = Path(f'data/kaggle/submissions/{filename}.csv')
predictions.to_csv(path, index=False)